## Imports

In [45]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [46]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [47]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [48]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [49]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [50]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [51]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [52]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,The W South Beach\r2201 Collins Ave 1228\rMiam...,2/10/2023,77,"$3,950,000.00",1239,"$3,188.05",Monica Nishi,Miami Connections Realty LLC,Linwood Featherston,Sterling Equity Realty LLC,The W South Beach 2201 Collins Ave 1228 Miami ...,The W South Beach,2201 Collins Ave 1228 Miami Beach,"(25.7973813, -80.1271903)",25.797381,-80.127190
2,Bristol Tower Condo\r2127 Brickell Ave 3101\rM...,2/10/2023,233,"$3,300,000.00",3065,"$1,076.67",Jeannie Montes de Oca,BHHS EWM Realty,Emma Garavito,BHHS EWM Realty,Bristol Tower Condo 2127 Brickell Ave 3101 Miami,Bristol Tower Condo,2127 Brickell Ave 3101 Miami,"(25.7522291, -80.1994471)",25.752229,-80.199447
3,Villa Harbour\r166 Harbor Dr 1A\rKey Biscayne,2/10/2023,52,"$2,850,000.00",1652,"$1,725.18",Shane Graber,Graber Realty Group LLC,Enzo Rosani,BARNES INT'L REALTY LLC,Villa Harbour 166 Harbor Dr 1A Key Biscayne,Villa Harbour,166 Harbor Dr 1A Key Biscayne,"(25.6997468, -80.1680305)",25.699747,-80.168031
4,Ritz Carlton Sunny Isles\r15701 Collins Ave 35...,2/8/2023,165,"$2,815,000.00",1447,"$1,945.40",Cassio Galiza PA,Podium Realty Group INC,Mihaela Moldoveanu,"Grand Realty of America, Corp.",Ritz Carlton Sunny Isles 15701 Collins Ave 350...,Ritz Carlton Sunny Isles,15701 Collins Ave 3503 Sunny Isles Beach,"(25.922045, -80.12199729999999)",25.922045,-80.121997
5,Icon Brickell\r465 Brickell Ave 4801\rMiami,2/9/2023,58,"$2,300,000.00",1825,"$1,260.27",Elizabeth Salomon,International Prestige Associates,Nicole Dwyer,EXP Realty LLC,Icon Brickell 465 Brickell Ave 4801 Miami,Icon Brickell,465 Brickell Ave 4801 Miami,"(25.7689974, -80.18854999999999)",25.768997,-80.188550
6,Monaco Yacht Club\r6800 Indian Creek Dr 10B\rM...,2/9/2023,117,"$2,250,000.00",1386,"$1,623.38",Steven Miles,Steven Miles PA,Alejandro Vasquez,Million and Up Realty,Monaco Yacht Club 6800 Indian Creek Dr 10B Mia...,Monaco Yacht Club,6800 Indian Creek Dr 10B Miami Beach,"(25.8523985, -80.1229903)",25.852398,-80.122990
7,Icon Bay Condo\r460 NE 28th St PH4101\rMiami,2/9/2023,1,"$2,180,000.00",2264,$962.90,Charles Celesia,Compass Florida LLC,Neila Rudden,One Sotheby's Int'l Realty,Icon Bay Condo 460 NE 28th St PH4101 Miami,Icon Bay Condo,460 NE 28th St PH4101 Miami,"(25.8036403, -80.1871142)",25.803640,-80.187114
8,Ocean Three\r18911 Collins Ave 2005\rSunny Isl...,2/10/2023,128,"$1,975,000.00",2864,$689.60,Adam Bursztein,Compass Florida LLC,Olena Zhuravska,New Level Corp,Ocean Three 18911 Collins Ave 2005 Sunny Isles...,Ocean Three,18911 Collins Ave 2005 Sunny Isles Beach,"(25.9508781, -80.1199875)",25.950878,-80.119987
9,900 Biscayne Bay\r900 Biscayne Blvd 2901\rMiami,2/7/2023,245,"$1,475,000.00",1694,$870.72,Domingo Bolet,"Xpress Realty, LLC",Esha Desaiy,Compass Florida LLC,900 Biscayne Bay 900 Biscayne Blvd 2901 Miami,,900 Biscayne Bay 900 Biscayne Blvd 2901 Miami,"(25.7834936, -80.19020909999999)",25.783494,-80.190209
10,0Parque Towers\r330 Sunny Isles Blvd 5-901\rSu...,2/10/2023,44,"$1,390,000.00",1892,$734.67,Chad Carroll,Compass Florida LLC,Anita Funtek,Vera Realty LLC,0Parque Towers 330 Sunny Isles Blvd 5-901 Sunn...,,0Parque Towers 330 Sunny Isles Blvd 5-901 Sunn...,"(25.9294643, -80.1259521)",25.929464,-80.125952


In [53]:
df.at[9,'building_name']=('900 Biscayne Bay')
df.at[9,'address_only']=('900 Biscayne Blvd 2901')

df.at[10,'building_name']=('Parque Towers')
df.at[10,'address_only']=('330 Sunney Isles Blvd 5-901')

In [54]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [55]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 5th - February 11th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r['building_name'],r['Sale Price'],r['days_on_market'],r['Listing Broker'],r['Buyer Agent'],r['price_per_sqft'],r['address_only'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[0]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[1]}{bold_end}{new_line}\
    Days on Market: {bold_start}{info[2]}{bold_end}{new_line}\
    Listing Agent: {bold_start}{info[3]}{bold_end}{new_line}\
    Buying Agent: {bold_start}{info[4]}{bold_end}{new_line}\
    Price sq ft: {bold_start}{info[5]}{bold_end}{new_line}\
    Address: {bold_start}{info[6]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [56]:
m.save('index.html')

## Data snagger

In [57]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [58]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [59]:
df['Sale Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['Sale Price'] = df['Sale Price'].str.replace(',','',regex=True)
df['Sale Price'] = pd.to_numeric(df['Sale Price'])

In [60]:
df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

In [61]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])

In [62]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [69]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['Sale Price'].idxmax()]['building_name']}, {df.loc[df['Sale Price'].idxmax()]['address_only']} | Price ${df.loc[df['Sale Price'].idxmax()]['Sale Price']:,.0f} | ${df.loc[df['Sale Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['Sale Price'].idxmax()]['Agent']} with {df.loc[df['Sale Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['Sale Price'].idxmax()]['Buyer Agent']} with {df.loc[df['Sale Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['Sale Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['Sale Price'].idxmin()]['building_name']}, {df.loc[df['Sale Price'].idxmin()]['address_only']} | Price ${df.loc[df['Sale Price'].idxmin()]['Sale Price']:,.0f} | ${df.loc[df['Sale Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['Sale Price'].idxmin()]['Agent']} with {df.loc[df['Sale Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['Sale Price'].idxmin()]['Buyer Agent']} with {df.loc[df['Sale Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['Sale Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['Sale Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['Sale Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['Sale Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['Sale Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
The W South Beach , 2201 Collins Ave 1228 Miami Beach | Price $3,950,000 | $3,188 psf | Listing agent: Monica Nishi with Miami Connections Realty LLC | Buyer's agent: Linwood Featherston with Sterling Equity Realty LLC | Days on market: 77
Least Expensive
Parque Towers, 330 Sunney Isles Blvd 5-901 | Price $1,390,000 | $735 psf | Listing agent: Chad Carroll with Compass Florida LLC | Buyer's agent: Anita Funtek with Vera Realty LLC | Days on market: 44
Highest Price Per Square Foot
The W South Beach , 2201 Collins Ave 1228 Miami Beach | Price $3,950,000 | $3,188 psf | Listing agent: Monica Nishi with Miami Connections Realty LLC | Buyer's agent: Linwood Featherston with Sterling Equity Realty LLC | Days on market: 77
Lowest Price Per Square Foot
Ocean Three , 18911 Collins Ave 2005 Sunny Isles Beach | Price $1,975,000 | $690 psf | Listing agent: Adam Bursztein with Compass Florida LLC | Buyer's agent: Olena Zhuravska with New Level Corp | Days on market: 128
Most Days on 

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [64]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [65]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_02112023
